**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function `act` takes as input the current state tensor $s$ and outputs an integer action $0 < a < \text{n_action}$.

The $\varepsilon$ corresponds to the exploitation/exploration rate. For instance, if $\varepsilon = 5\%$, then the algorithm will pick a random action with a probability $0.05$ and explore the state space, or exploit the current policy to get the currently-best action with probability $0.95$. 

This is an essential part of the algorithm since an algorithm without exploration could miss out on a lot of potentially rewarding states by only going through the states it already knows, while an algorithm without exploitation will return a random policy. This leads to the famous exploration-exploitation dilemma which consists in finding an appropriate $\varepsilon$ rate, as a trade-off between exploration and exploitation.



***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=60 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array `board` is of size $(N+4)\times(N+4)$, the additional border of size $2$ being used as padding for the $5\times 5$ state matrix. The board contains zeros everywhere, except on the locations of bonuses and maluses where the boards contains respectively $0.5$ and $-1.0$ (if there is both a bonus and a malus on one case during generation, then only the bonus is kept). Each time the agent goes on a case, the value of this case in the array becomes $0$. 

The `state` array is of the same size as `board`. It contains $0$s everywhere except at the position of the agent which is set to $1$, and in the padding border of size $2$ where it is set to $-1$.

These two arrays are then concatenated element-wise to form the state tensor. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

# we could also set epsilon to 1

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):        
        win = 0
        lose = 0
        
        done = False
        state = env.reset()
        while not done:
            action = agent.act(state, train=False)
            state, reward, done = env.act(action)
            if reward > 0:
                win += reward
            elif reward < 0:
                lose -= reward
                
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/12.0. Average score (-2.0)
Win/lose count 8.0/14.0. Average score (-4.0)
Win/lose count 6.5/10.0. Average score (-3.8333333333333335)
Win/lose count 9.5/11.0. Average score (-3.25)
Win/lose count 9.5/18.0. Average score (-4.3)
Win/lose count 9.0/17.0. Average score (-4.916666666666667)
Win/lose count 9.5/18.0. Average score (-5.428571428571429)
Win/lose count 12.0/17.0. Average score (-5.375)
Win/lose count 9.0/13.0. Average score (-5.222222222222222)
Win/lose count 8.0/9.0. Average score (-4.8)
Final score: -4.8


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




$\bullet$ By definition, 

$$Q^\pi(s,a) = E_{p^\pi}\left[\sum_{t \geq 0} \gamma^t r(s_t,a_t) \mid s_0=s,a_0=a\right]$$

Then,

\begin{align*}
Q^\pi(s,a) &= E_{p^\pi}\left[r(s_0,a_0) + \gamma \sum_{t \geq 0} \gamma^t r(s_{t+1},a_{t+1}) \mid s_0=s,a_0=a\right] \\
&= E_{(s',a') \sim p(.\mid s,a)}\left[r(s,a) + \gamma E_{p^\pi} \left[ \sum_{t \geq 0} \gamma^t r(s_{t+1},a_{t+1}) \mid s_1=s',a_1=a'\right] \right] \\
&= E_{(s',a') \sim p(.\mid s,a)}\left[r(s,a) + \gamma Q^\pi(s',a') \right]
\end{align*}

$\bullet$ We have $Q^*(s,a) = Q^{\pi^*}(s,a)$, and we assume that $\pi^*$ is chosen deterministic (which is possible if the state and action spaces are finite). Then,

\begin{align*}
Q^*(s,a) &= E_{(s',a') \sim \pi^*(.\mid s,a)}\left[r(s,a) + \gamma Q^*(s',a') \right] \\
&= E_{s' \sim \pi^*(.\mid s,a)}\left[r(s,a) + \gamma Q^*(s',\pi^*(s')) \right] \\
\end{align*}

and $\pi^*$ being optimal, $\pi^*(s')$ maximizes $Q^*(s',\cdot)$, thus 

$$Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]$$

$\bullet$ Since the goal is to find $Q^*$, we can aim at making the previous equality true. It can be rewritten as

$$E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a') - Q^{*}(s,a)] = 0$$

thus a plausible objective would be one that minimizes its norm with respect to $Q$:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()
        self.current = 0

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            # append to memory if it is not full yet
            self.memory.append(m)
        else:
            # if memory is full, rewrite over oldest element in memory
            self.memory[self.current] = m
            self.current = (self.current + 1) % self.max_memory

    def random_access(self):
        # return a random element of the memory
        idx = np.random.randint(len(self.memory))
        return self.memory[idx]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

I define a function to evaluate an agent (ie without exploration):

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):  # abstract class
    def __init__(self, grid_size, epsilon=0.1, memory_size=100, batch_size=16, n_state=2, **kwargs):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        Q = self.model.predict(np.expand_dims(s, axis=0))
        return np.argmax(Q)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s, ns, a, r, game_over = self.memory.random_access()
            
            input_states[i] = s
            target_q[i] = self.model.predict(np.expand_dims(s, axis=0))
            
            if not game_over:
                Q_ns = self.model.predict(np.expand_dims(ns, axis=0))
                target_q[i,a] = r + self.discount * np.max(Q_ns)
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

from keras.layers import Flatten    
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(64))        
        model.add(Activation('relu'))
        model.add(Dense(32))        
        model.add(Activation('relu'))
        model.add(Dense(16))        
        model.add(Activation('relu'))
        model.add(Dense(4))
        
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile(Adam(lr=lr), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.01, epsilon=0.2, memory_size=5*T, batch_size = 64)
train(agent, env, epoch=21, prefix='fc_train')

Epoch 000/021 | Loss 0.0059 | Win/lose count 5.5/11.0 (-5.5)
Epoch 001/021 | Loss 0.0034 | Win/lose count 3.5/6.0 (-2.5)
Epoch 002/021 | Loss 0.0050 | Win/lose count 10.0/7.0 (3.0)
Epoch 003/021 | Loss 0.0017 | Win/lose count 4.5/1.0 (3.5)
Epoch 004/021 | Loss 0.0018 | Win/lose count 4.5/6.0 (-1.5)
Epoch 005/021 | Loss 0.0036 | Win/lose count 5.5/1.0 (4.5)
Epoch 006/021 | Loss 0.0035 | Win/lose count 7.0/2.0 (5.0)
Epoch 007/021 | Loss 0.0044 | Win/lose count 13.0/8.0 (5.0)
Epoch 008/021 | Loss 0.0013 | Win/lose count 9.0/4.0 (5.0)
Epoch 009/021 | Loss 0.0029 | Win/lose count 11.0/4.0 (7.0)
Epoch 010/021 | Loss 0.0027 | Win/lose count 11.0/6.0 (5.0)
Epoch 011/021 | Loss 0.0009 | Win/lose count 8.0/2.0 (6.0)
Epoch 012/021 | Loss 0.0022 | Win/lose count 20.0/5.0 (15.0)
Epoch 013/021 | Loss 0.0003 | Win/lose count 15.0/2.0 (13.0)
Epoch 014/021 | Loss 0.0023 | Win/lose count 11.5/8.0 (3.5)
Epoch 015/021 | Loss 0.0006 | Win/lose count 17.0/8.0 (9.0)
Epoch 016/021 | Loss 0.0004 | Win/lose cou

In [13]:
HTML(display_videos('fc_train20.mp4'))

After just 20 training iterations, the FC agent is already doing pretty good ratio-wise.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [14]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        if 'filters' not in kwargs:
            kwargs['filters'] = [20, 10]
        
        model = Sequential()
        model.add(Conv2D(filters=kwargs['filters'][0], kernel_size=2, input_shape=(5,5,self.n_state)))
        model.add(Activation('relu'))
        model.add(Conv2D(filters=kwargs['filters'][1], kernel_size=2))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(4))        
        
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile(Adam(lr=lr), "mse")
        self.model = model

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.01, epsilon = 0.2, memory_size=5*T, batch_size = 64)
train(agent,env,epoch=21,prefix='cnn_train')

Epoch 000/021 | Loss 0.0028 | Win/lose count 2.5/5.0 (-2.5)
Epoch 001/021 | Loss 0.0004 | Win/lose count 2.0/2.0 (0.0)
Epoch 002/021 | Loss 0.0008 | Win/lose count 10.0/5.0 (5.0)
Epoch 003/021 | Loss 0.0013 | Win/lose count 9.5/3.0 (6.5)
Epoch 004/021 | Loss 0.0016 | Win/lose count 8.5/1.0 (7.5)
Epoch 005/021 | Loss 0.0009 | Win/lose count 7.0/4.0 (3.0)
Epoch 006/021 | Loss 0.0046 | Win/lose count 6.5/4.0 (2.5)
Epoch 007/021 | Loss 0.0009 | Win/lose count 1.5/4.0 (-2.5)
Epoch 008/021 | Loss 0.0016 | Win/lose count 9.0/2.0 (7.0)
Epoch 009/021 | Loss 0.0006 | Win/lose count 16.0/3.0 (13.0)
Epoch 010/021 | Loss 0.0005 | Win/lose count 19.5/5.0 (14.5)
Epoch 011/021 | Loss 0.0008 | Win/lose count 12.0/4.0 (8.0)
Epoch 012/021 | Loss 0.0005 | Win/lose count 15.0/10.0 (5.0)
Epoch 013/021 | Loss 0.0004 | Win/lose count 13.5/7.0 (6.5)
Epoch 014/021 | Loss 0.0015 | Win/lose count 16.5/6.0 (10.5)
Epoch 015/021 | Loss 0.0005 | Win/lose count 20.0/3.0 (17.0)
Epoch 016/021 | Loss 0.0006 | Win/lose co

In [16]:
HTML(display_videos('cnn_train20.mp4'))

Similarly, the CNN agent is doing very well after 20 training iterations. The ratios it achieves seem quite better than the ratios achieved by the FC agent.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs=10,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs=10,prefix='fc_test')

Test of the CNN
Win/lose count 11.0/0. Average score (11.0)
Win/lose count 3.0/0. Average score (7.0)
Win/lose count 5.5/0. Average score (6.5)
Win/lose count 5.5/0. Average score (6.25)
Win/lose count 15.0/0. Average score (8.0)
Win/lose count 1.5/0. Average score (6.916666666666667)
Win/lose count 13.5/0. Average score (7.857142857142857)
Win/lose count 2.5/0. Average score (7.1875)
Win/lose count 8.0/0. Average score (7.277777777777778)
Win/lose count 1.0/0. Average score (6.65)
Final score: 6.65
Test of the FC
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 4.5/2.0. Average score (2.0)
Win/lose count 1.5/0. Average score (1.8333333333333333)
Win/lose count 8.5/0. Average score (3.5)
Win/lose count 5.5/0. Average score (3.9)
Win/lose count 8.0/1.0. Average score (4.416666666666667)
Win/lose count 4.5/0. Average score (4.428571428571429)
Win/lose count 2.5/0. Average score (4.1875)
Win/lose count 5.0/0. Average score (4.277777777777778)
Win/lose count 2.5/0. Average score (4

In [18]:
HTML(display_videos('cnn_test9.mp4'))

In [19]:
HTML(display_videos('fc_test9.mp4'))

When removing the random exploration, we can see that both trained agent rarely go over maluses. However, they usually get stuck in an infinite loop between two adjacent cases. This is probably due to the fact that our final greedy policy is deterministic and sends two adjacent positions towards each other. We will try to fix it in question 10. Regarding the scores, the CNN agent seems to do a little bit better than the FC agent, but this is not very meaningful since the time after which the agents get stuck is quite random. 

In [20]:
for temp in [0.1, 0.5, 0.9]:
    env = Environment(grid_size=size, max_time=T,temperature=temp)
    print('Test of the CNN, temperature=' + str(temp))
    test(agent_cnn,env,epochs=5,prefix='cnn_test')
    print()

Test of the CNN, temperature=0.1
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 0.5/0. Average score (1.1666666666666667)
Win/lose count 3.0/0. Average score (1.625)
Win/lose count 0.5/1.0. Average score (1.2)
Final score: 1.2

Test of the CNN, temperature=0.5
Win/lose count 22.5/0. Average score (22.5)
Win/lose count 16.5/1.0. Average score (19.0)
Win/lose count 9.5/1.0. Average score (15.5)
Win/lose count 11.0/1.0. Average score (14.125)
Win/lose count 9.5/0. Average score (13.2)
Final score: 13.2

Test of the CNN, temperature=0.9
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 1.0/0. Average score (0.75)
Win/lose count 9.0/0. Average score (3.5)
Win/lose count 24.5/0. Average score (8.75)
Win/lose count 1.5/0. Average score (7.3)
Final score: 7.3



We can see that when we increase the temperature, the scores get higher. This is normal, since when both a bonus and a malus are generated on a case (in `env.reset()`), then only the bonus is kept. Thus with a high temperature, the map will be composed mostly of bonuses.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [21]:
def train_explore(agent,env,epoch,epsilon_discount=0.99,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, reward_real, game_over = env.act(action, train=True)

            # Update the counters
            if reward_real > 0:
                win = win + reward_real
            if reward_real < 0:
                lose = lose -reward_real

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        
        # Decreasing epsilon-greedy exploration
        agent.set_epsilon(epsilon_discount * agent.epsilon)

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.2f}/{:.2f} ({:.2f})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(
            grid_size=grid_size, max_time=max_time, temperature=temperature)
        self.malus_position = np.zeros((grid_size,grid_size))       

    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        reward_real = reward
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        if train:
            reward -= self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.1

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        if train:
            return state, reward, reward_real, game_over
        else:
            return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""
        self.malus_position = np.zeros((self.grid_size,self.grid_size))  

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.malus_position > 0,1] = 150
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b

In [22]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=5*T, batch_size = 64,n_state=3)
train_explore(agent, env, epoch=21, epsilon_discount=0.99, prefix='cnn_train_explore')

Epoch 001/021 | Loss 0.0209 | Win/lose count 9.50/12.00 (-2.50)
Epoch 002/021 | Loss 0.0073 | Win/lose count 20.00/1.00 (19.00)
Epoch 003/021 | Loss 0.0080 | Win/lose count 15.50/6.00 (9.50)
Epoch 004/021 | Loss 0.0048 | Win/lose count 18.50/7.00 (11.50)
Epoch 005/021 | Loss 0.0063 | Win/lose count 20.00/1.00 (19.00)
Epoch 006/021 | Loss 0.0092 | Win/lose count 23.50/2.00 (21.50)
Epoch 007/021 | Loss 0.0049 | Win/lose count 21.00/3.00 (18.00)
Epoch 008/021 | Loss 0.0047 | Win/lose count 23.00/2.00 (21.00)
Epoch 009/021 | Loss 0.0080 | Win/lose count 21.50/6.00 (15.50)
Epoch 010/021 | Loss 0.0062 | Win/lose count 26.50/2.00 (24.50)
Epoch 011/021 | Loss 0.0065 | Win/lose count 23.00/3.00 (20.00)
Epoch 012/021 | Loss 0.0126 | Win/lose count 24.00/9.00 (15.00)
Epoch 013/021 | Loss 0.0136 | Win/lose count 25.00/9.00 (16.00)
Epoch 014/021 | Loss 0.0062 | Win/lose count 24.00/9.00 (15.00)
Epoch 015/021 | Loss 0.0069 | Win/lose count 21.50/7.00 (14.50)
Epoch 016/021 | Loss 0.0093 | Win/lose co

In [24]:
HTML(display_videos('cnn_train_explore20.mp4'))

In [25]:
# Evaluation
test(agent,env,epochs=21,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore20.mp4'))

Win/lose count 22.5/4.0. Average score (18.5)
Win/lose count 24.0/3.0. Average score (19.75)
Win/lose count 27.0/3.0. Average score (21.166666666666668)
Win/lose count 17.5/8.0. Average score (18.25)
Win/lose count 22.0/2.0. Average score (18.6)
Win/lose count 17.5/5.0. Average score (17.583333333333332)
Win/lose count 16.5/2.0. Average score (17.142857142857142)
Win/lose count 20.0/4.0. Average score (17.0)
Win/lose count 22.0/9.0. Average score (16.555555555555557)
Win/lose count 25.0/4.0. Average score (17.0)
Win/lose count 24.0/5.0. Average score (17.181818181818183)
Win/lose count 31.5/1.0. Average score (18.291666666666668)
Win/lose count 21.0/5.0. Average score (18.115384615384617)
Win/lose count 23.0/5.0. Average score (18.107142857142858)
Win/lose count 26.0/3.0. Average score (18.433333333333334)
Win/lose count 18.5/3.0. Average score (18.25)
Win/lose count 26.5/2.0. Average score (18.61764705882353)
Win/lose count 23.0/3.0. Average score (18.694444444444443)
Win/lose count 2

In drew in green the cases that the agent has already explored, ie where he gets an exploration malus. 

The agent is playing much better than before. He does not get stuck in loops anymore, and collects lots of bonuses and not too many maluses.

I threw in a last training with more epochs, more initial exploration are more filters in the convolutional layers, and managed to improve the results a little more. 

In [26]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.01, epsilon = 0.5, memory_size=8*T, batch_size = 64,n_state=3, filters=[32, 16])
train_explore(agent, env, epoch=51, epsilon_discount=0.95, prefix='cnn_train_explore')

Epoch 001/051 | Loss 0.0319 | Win/lose count 6.50/10.00 (-3.50)
Epoch 002/051 | Loss 0.0092 | Win/lose count 11.00/12.00 (-1.00)
Epoch 003/051 | Loss 0.0078 | Win/lose count 13.00/11.00 (2.00)
Epoch 004/051 | Loss 0.0079 | Win/lose count 17.50/5.00 (12.50)
Epoch 005/051 | Loss 0.0057 | Win/lose count 16.00/13.00 (3.00)
Epoch 006/051 | Loss 0.0050 | Win/lose count 20.00/13.00 (7.00)
Epoch 007/051 | Loss 0.0057 | Win/lose count 15.00/11.00 (4.00)
Epoch 008/051 | Loss 0.0067 | Win/lose count 20.00/12.00 (8.00)
Epoch 009/051 | Loss 0.0078 | Win/lose count 18.00/10.00 (8.00)
Epoch 010/051 | Loss 0.0131 | Win/lose count 15.00/9.00 (6.00)
Epoch 011/051 | Loss 0.0053 | Win/lose count 23.00/6.00 (17.00)
Epoch 012/051 | Loss 0.0044 | Win/lose count 21.50/10.00 (11.50)
Epoch 013/051 | Loss 0.0050 | Win/lose count 22.00/12.00 (10.00)
Epoch 014/051 | Loss 0.0024 | Win/lose count 26.00/5.00 (21.00)
Epoch 015/051 | Loss 0.0043 | Win/lose count 19.50/13.00 (6.50)
Epoch 016/051 | Loss 0.0073 | Win/lose

In [27]:
# Evaluation
test(agent,env,epochs=31,prefix='cnn_optimized')

Win/lose count 26.5/5.0. Average score (21.5)
Win/lose count 24.5/4.0. Average score (21.0)
Win/lose count 23.0/8.0. Average score (19.0)
Win/lose count 25.0/3.0. Average score (19.75)
Win/lose count 20.0/4.0. Average score (19.0)
Win/lose count 17.5/7.0. Average score (17.583333333333332)
Win/lose count 26.0/3.0. Average score (18.357142857142858)
Win/lose count 24.5/1.0. Average score (19.0)
Win/lose count 25.0/1.0. Average score (19.555555555555557)
Win/lose count 29.0/2.0. Average score (20.3)
Win/lose count 26.5/3.0. Average score (20.59090909090909)
Win/lose count 23.0/6.0. Average score (20.291666666666668)
Win/lose count 18.5/3.0. Average score (19.923076923076923)
Win/lose count 30.0/3.0. Average score (20.428571428571427)
Win/lose count 28.0/2.0. Average score (20.8)
Win/lose count 26.0/3.0. Average score (20.9375)
Win/lose count 23.5/1.0. Average score (21.029411764705884)
Win/lose count 21.5/9.0. Average score (20.555555555555557)
Win/lose count 21.5/4.0. Average score (20.

An epoch where the agent collects all bonuses:

In [32]:
HTML(display_videos('cnn_optimized16.mp4'))

An epoch where the agent collects almost all bonuses without stepping on any malus:

In [33]:
HTML(display_videos('cnn_optimized24.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***